# Starting Landcover routine
    1) Reclassify existing landcover dataset(s)
    2) Derive majority categories for parcels
    3) Implement "burn in" routine to make crop parcels more discrete
    4) Cleanup misclassified crop pixels
    
The routine outlined above can be acheived using GIS software. The following code shows a Python implementation to derive starting landcover for a region. This code largely supersedes existing landcover processing tools in the data_processing.tools.landcover module.

*Author*: Jordan Dornbierer

In [ ]:
import os, sys
import numpy as np
import gdal
import json
from scipy import ndimage
import pandas as pd

local_path_to_repos = 'd:/Data/repos/python'
sys.path.insert(0, local_path_to_repos)
from data_processing.lib import utils
from data_processing.tools import landcover

## Required Datasets
    1) Dataset defining study area region and extents, typically region/background = 1/0.
    2) CDL, NLCD, or other landcover dataset that will be reclassified.
    3) Ownership parcels dataset. Each parcel should have a unique id.
    
**Note**: The landcover, parcel, and region datasets should conform in spatial projection, extent, and resolution. However, landcover and parcel datasets should not be masked for the region. This occurs during the burn-in routine and enables the retainment of edge-of-region parcels.

In [ ]:
# Set working dir
%cd z:/Working/Delaware_Basin/Starting_LULC_v2/
# %cd z:/Working/Northern_Lakes_Forest/starting_LULC

# Set input and output directorys
# in_dir = 'z:/Working/Delaware_Basin/Starting_LULC_v2/input'
# out_dir = 'z:/Working/Delaware_Basin/Starting_LULC_v2/output'
# in_dir = './starting_LULC/input'
# out_dir = './starting_LULC/output'

### Source CDL or NLCD

Get CDL or NLCD for region.  

Note: 
- "The 1997-2013 CDLs were recoded and re-released in January 2014 to better represent pasture and grass-related categories. A new category named Grass/Pasture (code 176) collapses the following historical CDL categories: Pasture/Grass (code 62), Grassland Herbaceous (code 171), and Pasture/Hay (code 181). This was done to eliminate confusion among these similar land cover types which were not always classified definitionally consistent from state to state or year to year and frequently had poor classification accuracies. This follows the recoding of the entire CDL archive in January 2012 to better align the historical CDLs with the current product. For a detailed list of the category name and code changes, please visit the Frequently Asked Questions (FAQ's) section at <http://www.nass.usda.gov/Research_and_Science/Cropland/sarsfaqs2.php"

- https://www.nass.usda.gov/Research_and_Science/Cropland/docs/CDL_2013_crosswalk.php

In [ ]:
# Region name.
# region_name = 'eco45iv'
region_name = 'delaware_basin'
# region_name = 'eco50iv'

# Get dataset defining study area region.
region_ds = gdal.Open('./input/{}_region.tif'.format(region_name))
region_array = region_ds.ReadAsArray()

year = 2008

if os.path.exists('./output_{}'.format(year)) == False:
    os.mkdir('./output_{}'.format(year))

# Get original CDL for the study area.
# Note: We won't mask to the actual region boundary until majority parcels have been derived
#  so that crop fields are preserved in their entirety (not cut) at the border.

# Regionally subset CDL
region_cdl_fn = '{}_cdl{}.tif'.format(region_name, year)
region_cdl_fp = './input/{}'.format(region_cdl_fn)

# CONUS ref
cdl_fp = 'z:/Working/National_Datasets/Cropland_Data_Layer/{0}_30m_cdls/{0}_30m_cdls.img'.format(year)
cdl_ds = gdal.Open(cdl_fp)
                      
if os.path.exists(region_cdl_fp):
    # Load from existing file.
    cdl_array = gdal.Open(region_cdl_fp).ReadAsArray()
else:
    # Clip from original CONUS CDL.
    cdl_array = utils.raster_to_array(cdl_ds, region_ds)
    
    # Write clipped CDL2018 to new raster if desired
    utils.array_to_raster(cdl_array, region_cdl_fp, region_ds, build_rat=True, cmap_ref=cdl_ds)

## Destination Color Schema
This can be derived from an existing raster dataset with the desired class-value : color schema.  
Or from a JSON file containing class-values mapped to colors.

In [ ]:
# From foresce/model/foresce_lib/geotiffs.py:
# Convert color hexidecimal string of RGB values to base10 tuple
def parse_color_tuple(color_data):
    if isinstance(color_data, (list, tuple)):
        return color_data

    color_data = color_data.lstrip('#')
    length = len(color_data)
    return tuple(int(color_data[i:i + length // 3], base=16) for i in range(0, length, length // 3))

In [ ]:
# Colormap can be provided by any existing dataset with same color schema
# cmap_ds = gdal.Open('z:/working/Delaware_Basin/Starting_LULC/CDL_2018_reclass.tif')

# Class names, values, colors
region_schema_fp = './input/{}_schema_with_color.json'.format(region_name)
with open(region_schema_fp, 'r') as fh:
    region_schema = json.load(fh)

# Create color table
ct = gdal.ColorTable()
for cls_val, cls_dict in region_schema.items(): #cmap_config['classes']:
    color_tuple = parse_color_tuple(cls_dict['color'])
    ct.SetColorEntry(int(cls_val), color_tuple)

print(region_schema)

## 1) Reclassify
Reclassify NLCD or CDL to get a landcover dataset in the desired classification schema.

The routine below uses dictionary of key: value pairs to map old landcover values to new.

The recalssification schema can be created in the Python console or Jupyter Notebook, or loaded from a JSON file or CSV.
Here, we load from an existing JSON file.  

We assess the application of the reclass schema and make adjustments as necessary before creating the reclassified dataset.

In [ ]:
cdl_ds.GetDescription()

In [ ]:
# Get Raster Attribute Table (rat) from CDL.
cdl_bnd = cdl_ds.GetRasterBand(1)
cdl_rat = cdl_bnd.GetDefaultRAT()  # Raster Attribute Table
cdl_rat_fields = {i: cdl_rat.GetNameOfCol(i) for i in range(cdl_rat.GetColumnCount())}
print(cdl_rat_fields)

In [ ]:
# Get class names from appropriate column/field. See above. This can change depending on vintage.
# Note: these are indexed to class value i.e. class_names[class_val] returns the value's corresponding name.
cdl_class_names = cdl_rat.ReadAsArray(4)

# We can examine the from-value, name, and pixel count.
# Note: categories with high pixel counts might be considered for their own class in the new schema.
# print(list(zip(remaining_from_vals, cdl2018_class_names[remaining_from_vals], remaining_counts)))

# Examine CDL class counts for the region.
cdl_vals, cdl_counts = np.unique(cdl_array[region_array!=0], return_counts=True)

In [ ]:
cdl_df = pd.DataFrame(index=cdl_vals)
cdl_df['names'] = cdl_class_names[cdl_vals]
cdl_df['counts'] = cdl_counts
cdl_df['perc'] = np.round(cdl_counts/(region_array!=0).sum(), 4)

In [ ]:
# Examine highest count categories.
sort_index = cdl_df['counts'].sort_values(ascending=False).index
cdl_df.loc[sort_index].to_csv('./input/{}_cdl{}_counts.csv'.format(region_name, year))
cdl_df.loc[sort_index]

In [ ]:
# Each key in the recode/reclass dictionary is a single to-class value.
# The key's item is a list of one or more from-class values.
reclass_schema_fp = './input/{}_cdl_recode.json'.format(region_name)
with open(reclass_schema_fp, 'r') as fh:
    reclass_schema = json.load(fh)

print(reclass_schema)

In [ ]:
# Note, above, that to-class 8 will catch all remaining from_class values that are not explicitly assigned a new class.
# Check to ensure these categories are all crops before finishing the reclassification.
# Edit reclassification JSON as necessary.

# Assemble all recode from-vals.
recode_vals = []
for from_vals in reclass_schema.values():
    if isinstance(from_vals, list):
        recode_vals.extend(from_vals)
        
catch_all = [val for val in cdl_vals if val not in recode_vals]
cdl_df.loc[catch_all].sort_values('counts', ascending=False)

In [ ]:
# Compute counts for reclassified.

# Initialize reclassified DataFrame
reclass_vals = [int(val) for val in reclass_schema]
reclass_df = pd.DataFrame(index=reclass_vals)

# Initialize columns
reclass_df['names'] = ''
reclass_df['counts'] = 0

for val in reclass_df.index:
    reclass_df.loc[val, 'names'] = region_schema[str(val)]['name']
    from_vals = reclass_schema[str(val)]
    
    # Account for catch-all category
    if isinstance(from_vals, list) == False:
        from_vals = catch_all
        
    # Sum from-val counts for all from-classes going to given to-class    
    mask = np.in1d(cdl_vals, from_vals)
    reclass_df.loc[val, 'counts'] = cdl_counts[mask].sum()

# Compute %
reclass_df['perc'] = np.round(reclass_df['counts']/(region_array!=0).sum(), 4)

# Examine reclassified counts. Edit schema if necessary and recompute.
reclass_df

In [ ]:
# Create reclassified dataset

reclass_fn = region_cdl_fn[:-4] + '_reclass.tif'
reclass_fp = './output_{}/{}'.format(year, reclass_fn)

if os.path.exists(reclass_fp):
    reclass_array = gdal.Open(reclass_fp).ReadAsArray()
    
else:

    # initialize reclassified landcover
    reclass_array = np.zeros(cdl_array.shape, dtype=np.uint8)

    # Reclassify all from-vals mapped to to-vals as defined in the reclassify schema
    for to_val, from_vals in reclass_schema.items():
        if isinstance(from_vals, list):
            for from_val in from_vals:
                reclass_array[cdl_array == from_val] = int(to_val)

    # If all the remaining classes can sensibly be converted to class 8 "Other Crops" we can proceed.
    # Note: do not convert 0: "Background"
    remaining_mask = (reclass_array == 0) & (cdl_array != 0)
    reclass_array[remaining_mask] = 8

    # Write to raster dataset if desired.
    utils.array_to_raster(reclass_array, reclass_fp, region_ds, build_rat=True, cmap_ref=ct)

## 2) Majority Category Parcels
Use existing parcels dataset and the reclassified landcover dataset to derive majority categories at the parcel level for the region.

The steps below preserve parcels at the edge of the region.

Note: previously multiple applications (x4) of Majority Filter function in ArcMap (neighbors=4, replacement_thresh='half') was used to clean up parcels. However, has been found to eliminate small parcels, which is problematic for urban decline.

In [ ]:
# Create 2D index of unique parcels using scipy's ndimage module to iterate through parcels.

# Note: each item in the index is a tuple of slices bounding the 2D parcels
# Each tuple's positional index, i, corresponds to the unique parcel id i + 1
# E.g. index[123] returns the tuple for parcel with id = 124.

# First make parcel NoData = 0, as the find_objects() function expects background = 0.
parcels_array[parcels_array == parcel_nodata] = 0
index = np.array(ndimage.find_objects(parcels_array))  # list -> array for fancy indexing below

# Get unique parcel ids for the region and remove nodata = 0 from ids.
region_ids = np.unique(parcels_array[region_array != 0])
if 0 in region_ids:
    region_ids = region_ids[region_ids != 0]

# Create dictionary with parcel id: bounding box slices pairs for only the given region,
# excluding parcels outside the region.
# Note: using numpy fancy indexing much more efficinet than checking each parcel id for membership in region_ids.
region_index = dict(zip(region_ids, index[region_ids - 1]))

In [ ]:
# Get or supply list of crop class values
crop_vals = [1, 2, 3, 4, 5, 6, 7, 8]

#### Update 3/10/20
Prioritizing crops in majority parcels workflow.  
Current minimum criteria for parcel to be assigned to crop is 25%.

In [ ]:
# Get dataset of parcels for region.
# parcels_filename = 'delaware_basin_clu_hsin_parcels_maj_filt_x4.tif'
# parcels_filename = 'delaware_basin_clu_yanroy_hsin_parcels_majfilt_x4.img'  # TODO: BigTiff issue
# parcels_filepath = os.path.join(in_dir, parcels_filename)

parcels_filepath = './input/delaware_basin_clu_hsin_parcels.tif'
parcels_ds = gdal.Open(parcels_filepath)

# Here, Arc's Majority Filter function (# neighbors = 4, replacement thresh = half) was run 4x to clean up parcel edges.
# This is optional.

# Get parcels nodata value and array.
parcel_nodata = parcels_ds.GetRasterBand(1).GetNoDataValue()
parcels_array = parcels_ds.ReadAsArray()

In [ ]:
maj_cat_fn = '{}_{}_majority_categories.tif'.format(region_name, year)
maj_cat_fp = './output_{}/{}'.format(year, maj_cat_fn)

if os.path.exists(maj_cat_fp):
    maj_array = gdal.Open(maj_cat_fp).ReadAsArray()

else:
    # Initialize parcel majority dataset
    maj_array = np.zeros(cdl_array.shape, dtype=np.uint8)

    # Iterate through each parcel of the region. Using bounding box (bb) to isolate parcel pixels.
    for parcel_id, bb in region_index.items():

        if bb is None:
            continue
        
        # Initialize
        maj_found = False

        # Convert to tuple
        bb = tuple(bb)

        parcel_box = parcels_array[bb]

        # Isolate parcel pixels inside bounding box
        bb_mask = parcel_box == parcel_id

        # Optionally apply size criteria.
        if bb_mask.sum() > 2878:  # 2878 30m pixels is ~1 section
            continue

        # Find all category values for parcel
        parcel_vals = reclass_array[bb][bb_mask]  # 2D -> 1D
        unique, counts = np.unique(parcel_vals, return_counts=True)
                
        # Find crop category values for parcel
        crop_idx = np.in1d(unique, crop_vals)
        
        # If parcel constains crops attempt to prioritize crop categories over others
        if crop_idx.sum() > 0:
            unique_crop = unique[crop_idx]
            counts_crop = counts[crop_idx]
            if counts_crop.max()/parcel_vals.size >= .25:
                # Set unique, counts to crops
                unique = unique_crop
                counts = counts_crop

        # Determine max val for parcel, account for potential tying values
        max_vals = unique[counts == counts.max()]
        max_val = np.random.choice(max_vals)

        # Set majority value in maj_array
        maj_array[bb][bb_mask] = max_val

    # Write majority category parcels raster dataset if desired.
    utils.array_to_raster(maj_array, maj_cat_fp, region_ds, build_rat=True, cmap_ref=ct, nodata=0)

## 3) Apply burn-in routine
Here the burn-in schema is saved in a JSON file.

Values found under the "field_vals" are crop category values from the reclassified landcover dataset.

Values under "burn_all" define categories that should be burned back in everywhere, unaltered from the reclassified landcover dataset.

The "burn_some" key corresponds with a nested dictionary of to-val: from-vals. This enables burning in to be controlled at the class level if desired. For each to-class all classes in the from-vals are allowed to burn in.  

For Delaware Basin no "burn_some" values are specified.


In [ ]:
# Load the burn-in schema from JSON

burn_schema_fp = './input/delaware_basin_burn_schema.json'
with open(burn_schema_fp, 'r') as fh:
    burn_schema = json.load(fh)

print(burn_schema)

In [ ]:
burned_fn = reclass_fn[:-4] + '_maj_cat_burn_schema.tif'
burned_fp = './output_{}/{}'.format(year, burned_fn)

if os.path.exists(burned_fp):
    burned_array = gdal.Open(burned_fp).ReadAsArray()
    
else:

    # Initialize burned-in array by copying the reclassified landcover
    burned_array = reclass_array.copy()

    # Burn in region boundary
    burned_array[region_array == 0] = 0

    # Burn in majority parcels for "field_vals"
    # Note: this extends parcels beyond region boundary if necessary.
    for field_val in burn_schema['field_vals']:
        burned_array[maj_array == field_val] = field_val

    # Burn from-to specified by "burn_some"
    for from_val, to_vals in burn_schema['burn_some'].items():
        from_mask = burned_array == from_val
        for to_val in to_vals:
            to_mask = reclass_array == to_val
            burned_array[from_mask & to_mask] = to_val

    # Burn to-classes from "burn_all"
    from_mask = burned_array != 0
    for to_val in burn_schema['burn_all']:
        to_mask = reclass_array == to_val
        burned_array[from_mask & to_mask] = to_val
    
    # Write to raster
    utils.array_to_raster(burned_array, burned_fp, region_ds, build_rat=True, cmap_ref=ct, nodata=0)

## 4) Clean up aberrant crop pixels
Identify pixels that are classified among crops in the new landcover dataset but overlay parcels with majority categories that are not crop.

These pixels can be converted to the underlying majority parcel type or to the nearest natural vegetation class.

**Note**: The nearest dataset can be produced from any landcover dataset and a list of class values. However, the generation of nearest datasets using Euclidean distance may cause MemoryErrors for large regions.

In [ ]:
# Examine misclassified pixels before correcting
misclass_fp = './output_{0}/{1}_{0}_misclass_crop.tif'.format(year, region_name)

if os.path.exists(misclass_fp):
    misclass_array = gdal.Open(misclass_fp).ReadAsArray()
    
else:

    # Initialize
    misclassified = np.zeros(burned_array.shape, np.uint8)

    # Majority categories NoData mask
    maj_nodata_mask = maj_array == 0

    # Isolate crop pixels outside their crop-majority parcels (where parcels exist)
    for crop_val in crop_vals:
        maj_mask = maj_array == crop_val
        burned_mask = burned_array == crop_val
        misclass_mask = burned_mask & ~maj_mask & ~maj_nodata_mask
        misclassified[misclass_mask] = crop_val
        
    # Write to raster
    utils.array_to_raster(misclassified.astype(np.uint8), misclass_fp, region_ds, cmap_ref=ct, build_rat=True)

In [ ]:
# Examine misclassified pixels before correcting
miss_vals, miss_counts = np.unique(burned_array[misclassified!=0], return_counts=True)
all_vals, all_counts = np.unique(burned_array, return_counts=True)
miss_idx = np.in1d(all_vals, miss_vals)

# Populate Pandas Dataframe
df = pd.DataFrame(index=miss_vals, columns=['name', 'misclassified', 'total', 'perc'])
df.misclassified = miss_counts
df.total = all_counts[miss_idx]
df.perc = df.misclassified / df.total * 100

# Add names to 'name' column
for val in region_schema:
    if int(val) in df.index:
        df.loc[int(val), 'name'] = region_schema[val]['name']

# Show the DataFrame
df

In [ ]:
cleanup_fn = burned_fn[:-4] + '_cleanup.tif'
cleanup_fp = './output_{}/{}'.format(year, cleanup_fn)

if os.path.exists(cleanup_fp):
    cleanup_array = gdal.Open(cleanup_fp).ReadAsArray()
    
else:

    # Initialize cleanup array by copying burn-in array.
    cleanup_array = burned_array.copy()

    # Proceed to convert misclassified crop pixels to nat veg nearest or majority parcel class if it makes sense to do so.
    # cleanup_array[misclassified_mask] = nv_nearest_array[misclassified_mask]
    cleanup_array[misclassified != 0] = maj_array[misclassified != 0]

    # Write to raster
    utils.array_to_raster(cleanup_array, cleanup_fp, region_ds, build_rat=True, cmap_ref=ct, nodata=0)

In [ ]:
# # Alternatively could assign misclassified pixels to nat-veg nearest.

# # Get nat veg classes from reclassification schema.
# nv_vals = [12, 13, 14, 15, 16, 17, 18, 19]

# # Load existing or derive natural vegetation nearest dataset.
# nv_nearest_fn = reclass_fn[:-4] + '_nv_nearest.tif'
# nv_nearest_fp = './output/{}'.format(nv_nearest_fn)

# if os.path.exists(nv_nearest_fp):
#     nv_nearest_array = gdal.Open(nv_nearest_fp).ReadAsArray()
# else:
#     nv_nearest_array = landcover.get_nearest(reclass_array, nv_vals)
    
#     # Write to file to avoid recalculating.
#     utils.array_to_raster(nv_nearest_array, nv_nearest_fp, region_ds, cmap_ref=ct, nodata=0)

### Assess Change

In [ ]:
# Compute counts for reclassified and cleaned-up
# Note: masking w/ cleanup_array retains whole fields at boundary
# reclass_array = gdal.Open('./output/delaware_basin_cdl2018_reclass.tif').ReadAsArray()
# cleaned_array = gdal.Open('./output/delaware_basin_cdl2018_reclass_maj_cat_burn_schema_cleanup.tif').ReadAsArray()

mask = cleanup_array != 0
unique_rcls, counts_rcls = np.unique(reclass_array[mask], return_counts=True)
unique_clnd, counts_clnd = np.unique(cleanup_array[mask], return_counts=True)

In [ ]:
print(reclass_vals)
print(ur)
print(cr)
print(uc)

In [ ]:
# Initialize DataFrame with reclassified index
change_df = pd.DataFrame(index=reclass_vals)

# Add class names using region schema
change_df['names'] = [region_schema[str(val)]['name'] for val in reclass_vals]

# Add reclassified counts
idx_mask = np.in1d(reclass_vals, unique_rcls)
count_mask = np.in1d(unique_rcls, reclass_vals)
change_df['reclass'] = 0  # initialize
change_df.loc[idx_mask, 'reclass'] = counts_rcls[count_mask]

# Add burned-in/cleaned-up counts
idx_mask = np.in1d(reclass_vals, unique_clnd)
count_mask = np.in1d(unique_clnd, reclass_vals)
change_df['cleaned'] = 0
change_df.loc[idx_mask, 'cleaned'] = counts_clnd[count_mask]

# Comparison metrics
change_df['diff'] = change_df['cleaned'] - change_df['reclass']
change_df['perc_change'] = change_df['diff']/change_df['reclass'] * 100

# Write to csv
change_df.to_csv('./output_{0}/{1}_{0}_reclass_and_cleaned_histograms.csv'.format(year, region_name))

In [ ]:
cdl_ds.GetDescription()